In [65]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [66]:
train_data = np.genfromtxt("0000000000002419_training_ccpp_x_y_train.csv", delimiter = ',')
x_train = train_data[:, 0:train_data.shape[1]-1]
y_train = train_data[:, train_data.shape[1]-1]
x_test = np.genfromtxt("0000000000002419_test_ccpp_x_test.csv", delimiter = ",")

In [67]:
df = pd.DataFrame(x_train)
N = x_train.shape[1]
count =1
for i in range(N):
    for j in range(i, N):
        df[count] = df[df.columns[i]] * df[df.columns[j]]
        count += 1

for i in range(N):
    for j in range(i, N):
        for k in range(j, N):
            df[count] = df[df.columns[i]]*df[df.columns[j]]*df[df.columns[k]]
            count += 1

x_train = df.values

# adding dummy data to testing data
df = pd.DataFrame(x_test)
N = x_test.shape[1]
count =1
for i in range(N):
    for j in range(i, N):
        df[count] = df[df.columns[i]] * df[df.columns[j]]
        count += 1

for i in range(N):
    for j in range(i, N):
        for k in range(j, N):
            df[count] = df[df.columns[i]]*df[df.columns[j]]*df[df.columns[k]]
            count += 1

x_test = df.values

In [68]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [69]:
def step_gradient(x, y, learning_rate, m):
    M = x.shape[0]
    m_slope = (-2/M)*(((y - (x*m).sum(axis=1)).reshape(-1, 1))*x).sum(axis=0)
    m = m - learning_rate * m_slope
    return m

In [70]:
def generic_gd(x, y, learning_rate, num_iterations):
    m = np.zeros(x.shape[1])
    for i in range(num_iterations):
        m = step_gradient(x, y, learning_rate, m)
        #print(i, "Cost :", cost(x, y, m))
    return m

In [71]:
def cost(x, y, m):
    return ((y - (x*m).sum(axis=1)) ** 2).mean()

In [72]:
def run(x, y):
    df = pd.DataFrame(x)
    df[x.shape[1]] = 1
    x = df.values
    learning_rate = 0.009
    num_iterations = 1000
    m = generic_gd(x, y, learning_rate, num_iterations)
    print(m)
    return m
m = run(x_train, y_train)

[-1.07124629e+01 -6.11060503e+00 -2.81486494e+00 -7.32120739e-01
  4.15919256e-01 -7.32120739e-01  4.15919256e-01  8.96906596e-01
  8.96906596e-01  9.21618830e-01  6.44791462e-01 -2.81486494e+00
 -7.32120739e-01  4.15919256e-01  8.96906596e-01  4.15919256e-01
  8.96906596e-01  9.21618830e-01  9.21618830e-01  6.44791462e-01
  1.76802968e-01  8.96906596e-01  9.21618830e-01  6.44791462e-01
  6.44791462e-01  1.76802968e-01 -4.04637653e-01  1.76802968e-01
 -4.04637653e-01 -1.04527032e+00 -1.70754204e+00  4.54431287e+02]


In [73]:
def predict(x,m):
    return (x*m).sum(axis=1)

In [74]:
df = pd.DataFrame(x_test)
df[x_test.shape[1]] = 1
x_test = df.values
y_pred = predict(x_test, m)
print(y_pred)

[472.12824565 471.8653367  437.29587881 ... 442.16642059 444.00820369
 448.1320055 ]


In [75]:
np.savetxt('Predictions_ccpp_data.csv',y_pred, fmt = "%.8f")